Install library to scrape Reddit data

In [ ]:
!pip install asyncpraw



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.5/196.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.6 MB/s eta 0:00:00


Import the library and attempt to autheticate user creds

In [ ]:
import asyncpraw
# Authenticate your script using your Reddit app credentials
reddit = asyncpraw.Reddit(
    client_id="CLIENT_ID",
    client_secret="CLIENT_SECRET",
    password="PASSWORD",
    user_agent="Comment Extraction (by u/USERNAME)",
    username="USERNAME",
)

Let's look at the subreddit for Orange Theory Workout to get posts related to the community

In [ ]:
# Specify the subreddit you want to extract data from
subreddit_name = 'orangetheory'

# Get the subreddit object
subreddit = reddit.subreddit(subreddit_name)

# Extract data from the subreddit
hot_posts = subreddit.hot(limit=10)  # Extract top 10 posts
daily_posts = subreddit.new(limit=10)

# Iterate over the posts and print their titles and scores
for post in hot_posts:
    print('Title:', post.title)
    print('Score:', post.score)
# Specify the subreddit tag you want to search
tag = 'Early Intel'

# Get the top posts with the specified tag from the subreddit
intel_posts = subreddit.search(f'flair:"{tag}"', sort='top', limit=10)

# Print the titles, URLs, and bodies of the top posts
for post in intel_posts:
    print(f'Title: {post.title}')
    print(f'URL: {post.url}')
    print(f'Body: {post.selftext}')

# Store the extracted data in a CSV file or database as per your project requirements
# You can use Python's CSV module or a database library to accomplish this.


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Title: ** May 2023 Monthly Highlights **
Score: 151
Title: Daily Workout and General Chat for Thursday, 5/11/23
Score: 71
Title: Friday 12 May 2023 - 2G 60 minutes
Score: 98
Title: Fav motivational coach quotes
Score: 33
Title: Forgot my Heart Rate Monitor
Score: 31
Title: Any advantage to signing up for Mayhem other than a $15 t-shirt?
Score: 9
Title: Music Suggestions
Score: 4
Title: I Wish They Offered Cardio Classes
Score: 15
Title: Top notch customer service
Score: 45
Title: Running on OTF treadmills
Score: 1
Title: Friday 12 May 2023 - 2G 60 minutes
URL: https://www.reddit.com/r/orangetheory/comments/13eyzfd/friday_12_may_2023_2g_60_minutes/
Body: Switch template with low benches on the floor.

**Tread Block 1**
* 1 min base
* 1 min push
* 1 min AO
* 1 min WR
* 1 min push
* 1 min AO
* 1 min WR
* 30 sec AO
* Transition to floor

**Floor Block 1 - 7.5 minutes**
* Back to back superset:
    * 6 x reverse lunge (L)
    * 6 x step down reverse lunge (L)
    * 6 x reverse lunge (R)
   